In [1]:
#Imports:
from comet_ml import Experiment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import random
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,CategoricalNB
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords
import string
from sklearn import preprocessing
from sklearn.manifold import TSNE
import seaborn as sns
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn import svm
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from time import time
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
import optuna
sns.set(context='notebook', style='darkgrid', palette='colorblind', font='sans-serif', font_scale=1, rc=None)
matplotlib.rcParams['figure.figsize'] =[8,8]
matplotlib.rcParams.update({'font.size': 15})
matplotlib.rcParams['font.family'] = 'sans-serif'

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bartalisd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bartalisd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#Load data:
with open("../data_root.txt") as f:
    data_root_dir = f.read()
data_root_dir
train = pd.read_csv('%s/tweet_disaster/train.csv' % data_root_dir)
test = pd.read_csv('%s/tweet_disaster/test.csv' % data_root_dir)
#sub = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [3]:
#Analyse & clean data:
from data_analysis import analysis, data_cleaning
frame, train, vocabulary = analysis(train, test)
X, Y = data_cleaning(train, test)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bartalisd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bartalisd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/bartalisd/git/apricot_exp/experiments/tweet_disaster/data_analysis.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text_length'] = train.text.apply(lambda x: len(x.split()))


In [4]:
#Train-test cut:
from apricot_exp.evaluation import traintest
X_train, X_test, Y_train, Y_test = traintest(X,Y,0.3)

In [5]:
#TFIDF & transpose data:
max_num_features = 10000
from data_analysis import tfidf
features_t, features_test_t = tfidf(X_train, X_test, max_num_features)
print(features_t.shape)
print(features_test_t.shape)

(10000, 5264)
(10000, 2257)


In [6]:
from apricot_exp.comet_utils import *
api_key = load_api_key('../../comet_key.txt')

In [7]:
#Bare model:
model = LogisticRegression(max_iter=1000)
n = 9000
experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
experiment.log_parameters({
    'size': None,
    'function': None,
})
tfidf = TfidfVectorizer(sublinear_tf=True,max_features=n, min_df=1, norm='l2',  ngram_range=(1,2))
features = tfidf.fit_transform(X_train).toarray()
features_test = tfidf.transform(X_test).toarray()
print(features.shape, features_test.shape)
from apricot_exp.evaluation import train_eval
train_eval(model, features,  Y_train, features_test, Y_test, experiment)
experiment.end()

COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/c9fdb8d7a7fe47cfaf0de1d4357ed144



(5264, 9000) (2257, 9000)


COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Still uploading


In [8]:
from apricot_exp.func_tp import featureb, facilityloc, maxcov
from comet import suggest_config, extract_grid

In [9]:
from optuna.samplers import TPESampler, RandomSampler, GridSampler
from parameters import param, algo
parameters = param('featurebased')
search_alg = "RND"
algo = algo(search_alg, parameters)
    
from apricot_exp.evaluation import train_eval

def objective(trial):
    config = suggest_config(parameters, trial)
    Xtr_t, Xte_t = featureb(features_t, features_test_t, 100, config["function"], config["optimizer"])
    Xtr = Xtr_t.transpose()
    Xte = Xte_t.transpose()
    acc, pre, rec, roc = train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
    return roc

study = optuna.create_study(direction="maximize", sampler=algo)
study.optimize(objective, n_trials=3, n_jobs=1)
study.best_params

[I 2021-10-25 08:16:24,356] A new study created in memory with name: no-name-b3782ffd-5169-4bb8-8a97-84a42d718ba6
[W 2021-10-25 08:16:24,360] Trial 0 failed because of the following error: TypeError('featureb() takes 4 positional arguments but 5 were given',)
Traceback (most recent call last):
  File "/home/bartalisd/.conda/envs/py36/lib/python3.6/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-9-a7cb1b0e44f2>", line 11, in objective
    Xtr_t, Xte_t = featureb(features_t, features_test_t, 100, config["function"], config["optimizer"])
TypeError: featureb() takes 4 positional arguments but 5 were given


TypeError: featureb() takes 4 positional arguments but 5 were given

In [ ]:
for i in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]:
    experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
    experiment.log_parameters({
        'size': i,
        'function': 'featurebased'
    })
    n = i
    #start_time = time.time()
    Xtr_t, Xte_t = featureb(features_t, features_test_t, n, 'sqrt')
    #experiment.log_metric("running_time", time.time()-start_time)
    Xtr = Xtr_t.transpose()
    Xte = Xte_t.transpose()
    from apricot_exp.evaluation import train_eval
    train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
    experiment.end()

In [ ]:
for i in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]: 
    experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
    experiment.log_parameters({
        'size': i,
        'function': 'facilitylocation'
    })
    n = i
    Xtr_t, Xte_t = facilityloc(features_t, features_test_t, n, 'euclidean')
    Xtr = Xtr_t.transpose()
    Xte = Xte_t.transpose()
    from apricot_exp.evaluation import train_eval
    train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
    experiment.end()

In [ ]:
for i in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]: 
    n = i
    experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
    experiment.log_parameters({
        'size': i,
        'function': 'maxcoverage'
    })
    from apricot_exp.func_tp import maxcov
    Xtr_t, Xte_t = maxcov(features_t, features_test_t, n)
    Xtr = Xtr_t.transpose()
    Xte = Xte_t.transpose()
    from apricot_exp.evaluation import train_eval
    train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
    experiment.end()

In [ ]:
def randomtrain(X_tr, X_te, n):
    X_train_arr = X_tr.to_numpy()
    X_test_arr = X_te.to_numpy() 
    print(X_train_arr.shape, X_test_arr.shape)
    idxs = np.arange(X_tr.shape[0])
    np.random.shuffle(idxs)
    idx = idxs[:n]
    Xtr, Xte = X_train_arr[idx,:], X_test_arr[idx,:]
    return Xtr, Xte

In [ ]:
for j in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]: 
    for i in range(1,20):
        n = j
        experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
        experiment.log_parameters({
            'size': i,
            'function': 'random'
        })
        Xtr_t, Xte_t = randomtrain(features_t, features_test_t, n)
        Xtr = Xtr_t.transpose()
        Xte = Xte_t.transpose()
        from apricot_exp.evaluation import train_eval
        train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
        i = i+1
        experiment.end()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=20, max_depth=3)

In [ ]:
from apricot_exp.evaluation import train_eval
train_eval(model, features,  Y_train, features_test, Y_test, experiment)

## PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
from sklearn.preprocessing import StandardScaler
f = list(range(0, 10000))
x = features[:, f]
x = StandardScaler().fit_transform(x)
x = pd.DataFrame(x)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
x_pca = pca.fit_transform(x)
x_pca = pd.DataFrame(x_pca)
x_pca.head()

## Visualization

In [ ]:
import neptune
project = neptune.init(project_qualified_name='bartalisd/nlp-disaster-tweets',
             api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNmY5YjdlNTgtNjMxNC00ODQyLThiNGItNmM5NTBmZTM1MjA4In0=', 
            )

# Download experiments dashboard as pandas DataFrame
data = project.get_leaderboard()

data=data[data['tags'].map(len) > 1]
data = data.drop(['name','created', 'finished','owner','notes', 'running_time','size'], axis=1)
for i in ['channel_roc', 'channel_acc', 'channel_rec', 'channel_pre']:
    data[i]=data[i].astype("float64")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def barplot_res(dataframe, percentage, metric, y_name):
    selector = dataframe.apply(lambda x: percentage in x["tags"], axis=1)
    data_percentage=dataframe.loc[selector, :]
    data_percentage["tags"]=data_percentage["tags"].apply(lambda x: str(x))
    res_percentage=data_percentage.groupby(by='tags')[metric].mean().reset_index()
    res_percentage['model']=res_percentage['tags'].apply(lambda x: eval(x)[1])
    ax=sns.barplot(data=res_percentage, x='model', y=metric)
    plt.ylim((0.5, 0.9))
    ax.set(xlabel='Model', ylabel=y_name, title =percentage)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=60)
    return ax, res_percentage
def lineplot_res(dataframe, y_name, metric, res5k, mod):
    data=dataframe.copy()
    data['percentage']=data['tags'].apply(lambda x: x[1])
    data=data[~data['percentage'].apply(lambda x: 'd' in str(x))]
    data['model']=data['tags'].apply(lambda x: x[0])
    functions=['facilityloc_d', 'featurebased_d', 'maxcov_d', 'random_d', 'Tfidf']
    data=data[data['model'].isin(functions)]
    result=data.groupby(by=['model','percentage'])[metric].mean().reset_index()
    ax=sns.lineplot(data=result, x='percentage', y=metric, hue='model', palette=['yellow', 'b','r','forestgreen','blueviolet'  ])
    ax.set_title(mod,fontdict= { 'fontsize': 20, 'fontweight':'bold'})
    plt.xlabel('Number of features', fontsize= 15)
    plt.ylabel(y_name, fontsize= 15)
    plt.axhline(y=res5k, color='brown', linestyle='--')
    plt.setp(ax.get_legend().get_texts(), fontsize='15')
    plt.setp(ax.get_legend().get_title(), fontsize='20')
    return ax

In [ ]:
lineplot_res(data, 'Accuracy', 'channel_acc', 0.7984049623393886, 'NLP - Disasters tweets: Dim reduction' )

In [ ]:
lineplot_res(data, 'Precision', 'channel_pre', 0.8259162303664922, 'NLP - Disasters tweets: Dim reduction' )

In [ ]:
lineplot_res(data, 'Recall', 'channel_rec', 0.6621196222455404, 'NLP - Disasters tweets: Dim reduction' )

In [ ]:
lineplot_res(data, 'ROC AUC', 'channel_roc', 0.8622086211447222, 'NLP - Disasters tweets: Dim reduction' )